In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Configuration - E-commerce Event Streaming Pipeline
# MAGIC Direct Unity Catalog table creation (no intermediate paths)

# COMMAND ----------

# Event Hub Configuration
eh_namespace = "evhns-natraining.servicebus.windows.net"
eh_name = "evh-natraining-bijunew"
keyvault_scope = "dbx-ss-kv-natraining-2"
secret_name = "evh-natraining-read-write"
shared_access_key_name = "SharedAccessKeyToSendAndListen"

# Unity Catalog Configuration
catalog = "na-dbxtraining"
schema_bronze = "biju_bronze"
schema_silver = "biju_silver"
schema_gold = "biju_gold"

# Table Names (Full Qualified Names)
bronze_orders_table = f"`{catalog}`.{schema_bronze}.orders"
bronze_products_table = f"`{catalog}`.{schema_bronze}.products"
silver_table = f"`{catalog}`.{schema_silver}.order_details"
gold_brand_category_table = f"`{catalog}`.{schema_gold}.sales_by_brand_category"
gold_location_table = f"`{catalog}`.{schema_gold}.location_performance"
gold_product_table = f"`{catalog}`.{schema_gold}.product_performance"
gold_customer_table = f"`{catalog}`.{schema_gold}.customer_insights"

print("="*70)
print("CONFIGURATION")
print("="*70)
print(f"\nEvent Hub:")
print(f"  Namespace: {eh_namespace}")
print(f"  Event Hub: {eh_name}")
print(f"\nUnity Catalog:")
print(f"  Catalog: {catalog}")
print(f"  Bronze Schema: {schema_bronze}")
print(f"  Silver Schema: {schema_silver}")
print(f"  Gold Schema: {schema_gold}")
print("="*70)

# COMMAND ----------

# MAGIC %md
# MAGIC ## Get Secret from Key Vault

# COMMAND ----------

try:
    secret_value = dbutils.secrets.get(scope=keyvault_scope, key=secret_name)
    print("✓ Successfully retrieved secret from Key Vault")
    print(f"  - Secret name: {secret_name}")
    print(f"  - Scope: {keyvault_scope}")
    print(f"  - Secret length: {len(secret_value)} characters")
except Exception as e:
    print(f"✗ Error retrieving secret: {str(e)}")
    raise

# COMMAND ----------

# MAGIC %md
# MAGIC ## Build Event Hub Connection String

# COMMAND ----------

# Full connection string for Event Hub SDK
connection_string = (
    f"Endpoint=sb://{eh_namespace}/;"
    f"SharedAccessKeyName={shared_access_key_name};"
    f"SharedAccessKey={secret_value};"
    f"EntityPath={eh_name}"
)

print("✓ Event Hub connection string configured")
print(f"✓ Endpoint: sb://{eh_namespace}")
print(f"✓ Event Hub: {eh_name}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Create Schemas

# COMMAND ----------

# Create schemas if they don't exist
#spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{catalog}`.{schema_bronze}")
#spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{catalog}`.{schema_silver}")
#spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{catalog}`.{schema_gold}")

print("✓ Schemas created/verified:")
print(f"  - {catalog}.{schema_bronze}")
print(f"  - {catalog}.{schema_silver}")
print(f"  - {catalog}.{schema_gold}")

# COMMAND ----------

print("\n" + "="*70)
print("✓ CONFIGURATION COMPLETE")
print("="*70)
print("\nNext Steps:")
print("  1. Run 01_Data_Generator to send events")
print("  2. Run 02_Bronze_Layer to create orders & products tables")
print("  3. Run 03_Silver_Layer to join tables")
print("  4. Run 04_Gold_Layer to create aggregations")